# Record Linkage: Machine Learning Example

This example will walk you through a typical record linkage application using the recordlinkage package in Python. We will quickly address every step and end with setting up a machine learning model to classify record pairs. The steps we will be talking about are: 

1. The Principles of Record Linkage
1. Data Pre-processing
2. Indexing
3. Record Comparison
4. Classification 
5. Evaluation 

In [1]:
# load packages 
import recordlinkage 
import pandas
import sklearn
import numpy
import scipy
import jellyfish
from recordlinkage.datasets import load_febrl4
from recordlinkage.standardise import clean
from recordlinkage.standardise import phonetic

## The Principles of Record Linkage
The goal of record linkage is to determine if pairs of records describe the same identity. For instance, this is important for removing duplicates from a data source or joining two separate data sources together. Record linkage also goes by the terms data matching, merge/purge, duplication detection, de-duping, reference matching, entity resolution, disambiguation, co-reference/anaphora in various fields.

There are several approaches to record linkage that include 
    - exact matching, 
    - rule-based linking and 
    - probabilistic linking. 
- An example of **exact matching** is joining records based on social security number. This is what you already have done in SQL by joining tables by an unique identifier. 
- **Rule-based matching** involves applying a cascading set of rules that reflect the domain knowledge of the - records being linked. 
- In **probabilistic record linkages**, linkage weights are estimated to calculate the probability of a certain match.

In practical applications you will need record linkage techiques to combine information addressing the same entity that is stored in different data sources. Record linkage will also help you to address the quality of different data sources. For example, if one of your databases has missing values you might be able to fill those by finding an identical pair in a different data source. Overall, the main applications of record linkage are
    1. Merging two or more data files 
    2. Identifying the intersection of the two data sets 
    3. Updating data files (with the data row of the other data files) and imputing missing data
    4. Entity disambiguation and de-duplication

In [2]:
# use expample data that come with the package
dfA, dfB = load_febrl4()

In [3]:
# How does the first dataset look
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [4]:
# How does the second dataset look
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


## Data Pre-processing

Data pre-processing is an important step in a data anlysis project in general, in record linkage applications in particular. The goal of pre-processing is to transform messy data into a dataset that can be used in a project workflow.

Linking records from different data sources comes with different challenges that need to be addressed by the analyst. The analyst must determine whether or not two entities (individuals, businesses, geographical units) on two different files are the same. This determination is not always easy. In most of the cases there is no common uniquely identifing characteristic for a entity. For example, is Bob Miller from New Yor the same person as Bob Miller from Chicago in a given dataset? This detemination has to be executed carefully because consequences of wrong linkages may be substantial (is person X the same person as the person X on the list of identified terrorists). Pre-processing can help to make better informed decisions.

Pre-processing can be difficult because there are a lot of things to keep in mind. For example, data input errors, such as typos, misspellings, truncation, abbreviations, and missing values need to be corrected. Literature shows that preprocessing can improve matches. In some situations, 90% of the improvement in matching efficiency may be due to preprocessing. The most common reason why matching projects fail is lack of time and resources for data cleaning. 

The record linkage package comes with a "clean" add on which can help you with preprocessing. Recordlinkage’s standardise sub-module includes four built-in functions. Each of these functions tackle a different aspect of pre-processing.

In [5]:
## The clean() function is used to clean a single column within a data frame, where the column contains strings. 
## By default, clean() will turn all strings into lowercase and remove characters such as quotation marks & punctuation.

dfA["name_clean"] = clean(dfA["given_name"])
dfB["name_clean"] = clean(dfB["given_name"])

In [6]:
## The phonetic() function is used to convert strings into their corresponding phonetic codes. 
## This is particularly useful when comparing names where different possible spellings make it difficult to find 
## exact matches (Ex. Jillian and Gillian).

dfA["phonetic"] = phonetic(dfA["name_clean"], method="nysiis")
dfB["phonetic"] = phonetic(dfB["name_clean"], method="nysiis")

In [27]:
# How does the second dataset look
dfA.head(100)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,name_clean,phonetic
rec_id,,,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218,michaela,MACAL
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,courtney,CARTNY
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,charles,CARL
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,vanessa,VANAS
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,mikayla,MACAYL
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548,blake,BLAC
rec-1985-org,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690,NaN,NaN
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555,blakeston,BLACASTAN
rec-1473-org,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058,NaN,NaN


## Indexing

Indexing allows you to create candidate links, which basically means identifying pairs of data rows which might refer to the same real world entity. This is also called the comparison space (matrix). There are different ways to index data. The easiest is to create a full index and consider every pair a match. This is also the least efficient method, because we will be comparing every row of one dataset with every row of the other dataset.

If we had 10,000 records in data frame A and 100,000 records in data frame B, we would have 1,000,000,000 candidate links. You can see that comparing over a full index is getting inefficient when working with big data.

In [8]:
## Full Index
indexer = recordlinkage.FullIndex()
pairs = indexer.index(dfA, dfB)

In [9]:
## How many records do we have?
print (len(dfA), len(dfB), len(pairs))

(5000, 5000, 25000000)


We can do better if we actually include our knowledge about the data to eliminate bad link from the start. This can be done through blocking. This method includes only record pairs that are identical on one or more stored attributes of the person (or entity in general). The recordlinkage packages gives you multiple options for this. For example, you can block by using variables, which menas only links exactly equal on specified values will be kept. You can also use a neighbourhood index in which the rows in your dataframe are ranked by some value and python will only link between the rows that are closeby.

In [10]:
## Create Index with blocking
indexer = recordlinkage.BlockIndex(on=('given_name', 'surname'))
pairs = indexer.index(dfA, dfB)

print (len(pairs))

2574


Here, the set of candidate links is restricted to entries with the exact same name. Think carefully about the quality of your data when using this method. If you cannot guarantee exact matches between corresponding rows, you may exclude potential matches from your set of candidate links.

In [29]:
## However, blocking on name might be too restrictive
indexer = recordlinkage.BlockIndex(on='postcode')
pairs = indexer.index(dfA, dfB)

print (len(pairs))

28609


## Record Comparison

After you have created a set of candidate links, you’re ready to begin comparing the records associated with each candidate link. In recordlinkage you must initiate a Compare object prior to performing any comparison functionality between records. This object stores both dataframes, the candidate links, and a vector containing comparison results. Further, the Compare object contains the methods for performing comparisons. The code block below initializes the comparison object.

In [12]:
## Initializes the comparison object
compare_cl = recordlinkage.Compare()

Currently there are five specific comparison methods within recordlinkage: Compare.exact(), Compare.string(), Compare.numeric(), Compare.geo(), and Compare.date(). The Compare.exact() method is simple: if two values are an exact match a comparison score of 1 is returned, otherwise 0 is retured. The Compare.string() method is a bit more complicated and generates a score based on well known string-comparison algorithms (for this example Levenshtein or Jaro Winkler).

In [13]:
compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold=0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

In [14]:
## The comparing of record pairs starts when the compute method is called. 
## All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.

features = compare_cl.compute(pairs, dfA, dfB)
features.head()

given_name  surname  date_of_birth  suburb  \
rec_id       rec_id                                                       
rec-1070-org rec-1070-dup-0         1.0      0.0              1       0   
             rec-1124-dup-0         0.0      0.0              0       0   
rec-1124-org rec-1070-dup-0         0.0      0.0              0       0   
             rec-1124-dup-0         1.0      1.0              1       1   
rec-1016-org rec-4695-dup-0         0.0      0.0              0       0   

                             state  address_1  
rec_id       rec_id                            
rec-1070-org rec-1070-dup-0      0        1.0  
             rec-1124-dup-0      1        0.0  
rec-1124-org rec-1070-dup-0      0        0.0  
             rec-1124-dup-0      1        1.0  
rec-1016-org rec-4695-dup-0      0        0.0

In [15]:
features.head()

given_name  surname  date_of_birth  suburb  \
rec_id       rec_id                                                       
rec-1070-org rec-1070-dup-0         1.0      0.0              1       0   
             rec-1124-dup-0         0.0      0.0              0       0   
rec-1124-org rec-1070-dup-0         0.0      0.0              0       0   
             rec-1124-dup-0         1.0      1.0              1       1   
rec-1016-org rec-4695-dup-0         0.0      0.0              0       0   

                             state  address_1  
rec_id       rec_id                            
rec-1070-org rec-1070-dup-0      0        1.0  
             rec-1124-dup-0      1        0.0  
rec-1124-org rec-1070-dup-0      0        0.0  
             rec-1124-dup-0      1        1.0  
rec-1016-org rec-4695-dup-0      0        0.0

In [16]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,28609.000000,28609.000000,28609.000000,28609.000000,28609.000000,28609.000000
mean,0.122619,0.130973,0.131322,0.109581,0.327694,0.126813
std,0.328005,0.337376,0.337758,0.312372,0.469381,0.332769
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Classification

Now we have to decide which records belong to one person. We can do this pretty simple, but we can also use machine learning methods to classify records.

In [17]:
## Simple Classification: Check for how many attributes records are identical by summing the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     1576
5.0     1656
4.0      745
3.0      207
2.0      101
1.0     5611
0.0    18713
dtype: int64

In [18]:
matches = features[features.sum(axis=1) > 3]
print(len(matches))

3977


Now let's do this with a machine learning classifier. Keep in mind that most classifiers can not handle comparison vectors with missing values. To prevent issues with the classification algorithms. Supervised learning algorithms do need training data. Training data is data for which the true match status is known for each comparison vector. In the example in this section, we consider that the true match status of the first 5000 record pairs of our data is known.

In [19]:
## Generate Training Data and index
ml_pairs = matches[0:1000]
ml_matches_index = ml_pairs.index & pairs 

The Naive Bayes classifier is a probabilistic classifier. The probabilistic record linkage framework by Fellegi and Sunter (1969) is the most well-known probabilistic classification method for record linkage. Later, it was proved that the Fellegi and Sunter method is mathematically equivalent to the Naive Bayes method in case of assuming independence between comparison variables.

In [20]:
## Train the classifier
nb = recordlinkage.NaiveBayesClassifier()
nb.learn(ml_pairs, ml_matches_index)

## Predict the match status for all record pairs
result_nb = nb.predict(matches)

## Predict probability for record to be a match
prob_nb = nb.prob(matches)

In [21]:
## Check header
prob_nb.head()

rec_id        rec_id        
rec-1124-org  rec-1124-dup-0    1.0
rec-1016-org  rec-1016-dup-0    1.0
rec-4695-org  rec-4695-dup-0    1.0
rec-4158-org  rec-4158-dup-0    1.0
rec-4100-org  rec-4100-dup-0    1.0
dtype: float64

## Evaluation

The last step is to evaluate the results of the record linkage.

In [22]:
## Confusion matrix
conf_nb = recordlinkage.confusion_matrix(pairs, result_nb, len(features))
conf_nb

array([[ 3977, 24632],
       [    0,     0]])

In [23]:
## Precision and Accuracy
precision = recordlinkage.precision(conf_nb)
accuracy = recordlinkage.accuracy(conf_nb)

In [24]:
print(precision)

1.0


In [25]:
print(accuracy)

0.139012198958


In [26]:
## The F-score for this classification is
recordlinkage.fscore(conf_nb)

0.24409255508500585